# BIT Detection Kit Prototyping Demo Notebook
The following notebook describes the steps taken and outputs obtained using Mask R-CNN for object detection.

## Setup
The following cells set up the libraries that are required for us to train and validate the model.

### Install Required Dependencies

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Install dependecies
!cd $ROOT_DIR && \
pip3 install -r requirements.txt && \
python3 setup.py install

### Import Packages

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log


from src import dkit
%matplotlib inline

### Configurations

Run one of the code blocks below to import and load the configurations to use.

In [ ]:
config = dkit.DkitConfig()
DATA_DIR = os.path.join(ROOT_DIR, "datasets")

### Download & Extract Dataset
We download the dataset file and extract the dataset.

In [ ]:
!wget -P $DATA_DIR https://s3.amazonaws.com/dkitdata/mrcnn/IKEA_FIXA.zip
!unzip $DATA_DIR/IKEA_FIXA.zip -d $DATA_DIR

In [6]:
!python3 dkit.py train --dataset=./IKEA_FIXA --weights=last

Using TensorFlow backend.
Weights:  last
Dataset:  ./IKEA_FIXA
Logs:  /home/ubuntu/Dkit/logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_b